In [3]:
import requests
import pandas as pd
import time

In [4]:
token = 'n0emNjRIkzyy4Z9Ii7HYYvCtani5AWT4T7HOcdrm'

paramaters = {'min_rank':'grand-champion','max_rank':'grand-champion','playlist':'ranked-doubles'}

head = {'Authorization': token}

metadata = ['id', 'link', 'rocket_league_id', 'replay_title', 'map_code',
            'map_name', 'playlist_id', 'playlist_name', 'duration', 'overtime',
            'overtime_seconds', 'season', 'season_type', 'date', 'date_has_tz',
            'visibility', 'created', 'min_rank.tier', 'min_rank.division',
            'min_rank.name', 'min_rank.id', 'max_rank.tier', 'max_rank.division',
            'max_rank.name', 'max_rank.id', 'uploader.steam_id', 'uploader.name',
            'uploader.profile_url', 'uploader.avatar']

In [44]:
def get_players_csv(id: str) -> pd.DataFrame:

    try:
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/players/{id}/{id}-players.csv', sep=';')
    except:
        print('Time Out')
        time.sleep(5)
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/players/{id}/{id}-players.csv', sep=';')

    return df.T

In [43]:
def get_teams_csv(id: str) -> pd.DataFrame:

    try:
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/teams/{id}/{id}-team-stats.csv', sep=';') # need to build in error handlding to the function here
    except:
        print('Time Out')
        time.sleep(5)
        df = pd.read_csv(f'https://ballchasing.com/dl/stats/teams/{id}/{id}-team-stats.csv', sep=';')
        
    return df.set_index('color').T

In [42]:
def add_team_data(df: pd.DataFrame) -> pd.DataFrame:

    blue_dict = {}
    orange_dict = {}
    
    for index, row in df.iterrows():
        
        time.sleep(4)
        teams_data = get_teams_csv(row['id'])
        
        blue_dict[index] = teams_data['blue']
        orange_dict[index] = teams_data['orange']

    df['blue_team'] = pd.Series(blue_dict)
    df['orange_team'] = pd.Series(orange_dict)
    
    return df

In [57]:
def add_player_data(df: pd.DataFrame) -> pd.DataFrame:

    bluep1_dict = {}
    bluep2_dict = {}
    orange1_dict = {}
    orange2_dict = {}
    
    for index, row in df.iterrows():
        
        time.sleep(4)
        player_data = get_players_csv(row['id'])
        
        bluep1_dict[index] = player_data[0]
        bluep2_dict[index] = player_data[1]
        orange1_dict[index] = player_data[2]
        orange2_dict[index] = player_data[3]

    df['blue_player1'] = pd.Series(bluep1_dict)
    df['blue_player2'] = pd.Series(bluep2_dict)
    df['orange_player1'] = pd.Series(orange1_dict)
    df['orange_player2'] = pd.Series(orange2_dict)
    
    return df

In [9]:
data = pd.DataFrame()

def new_data(criteria, auth, stuff) -> pd.DataFrame:

    api_request = requests.get('https://ballchasing.com/api/replays', params=criteria, headers=auth)
    infojson = api_request.json()

    df = pd.json_normalize(infojson, record_path=['list'], sep='.')

    df = df.drop(['blue.players','orange.players'], axis=1)

    for column_name in ['blue_team','blue_player1','blue_player2','orange_team','orange_player1','orange_player2']:
        df.insert(df.columns.size, column_name, pd.Series(None, dtype='object'))

    # add_team_data(df)
    # add_player_data(df)
    
    return df

**Data Cleaning** *OUTDATED*

* Data is received in a (trash) json format

* Everything is flattened at first except for the color.players column which still contains raw json text

* Each color of json text is flattened and then concatenated back into the main DataFrame

* Unneccesary columns are dropped

* Columns for individual players (4 total) and teams (2 total) are then made

* Data for individual players is downloaded and populated into a Series and placed into DataFrame cells

* Data for teams is downloaded and populated into a Series and placed into DataFrame cells

In [45]:
data = pd.concat([data, new_data(paramaters, head, metadata)]).reset_index(drop=True)
data.tail()

,id,link,rocket_league_id,replay_title,map_code,map_name,playlist_id,playlist_name,duration,overtime,...,uploader.avatar,blue.goals,orange.goals,overtime_seconds,blue_team,blue_player1,blue_player2,orange_team,orange_player1,orange_player2
95,38fc8f15-b45e-4939-968f-53f77e232443,https://ballchasing.com/api/replays/38fc8f15-b...,70E4F3D24488EFE78FB2B8ABC38C4501,2023-05-14.23.19 BipolarMoriarty Ranked Double...,underwater_p,Aquadome,ranked-doubles,Ranked Doubles,311,False,...,https://avatars.akamai.steamstatic.com/fef49e7...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,be6b5edd-c9a2-4fbf-812c-4d6483112d7d,https://ballchasing.com/api/replays/be6b5edd-c...,CA42512F4D0A42167D2B30871CC539E8,2023-05-14.17.19 Maximum_67 Ranked Doubles Loss,beach_night_p,Salty Shores (Night),ranked-doubles,Ranked Doubles,356,False,...,https://avatars.akamai.steamstatic.com/5c88471...,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,ce71a975-3dbb-470b-88ec-da2462f55b5f,https://ballchasing.com/api/replays/ce71a975-3...,75ADB7894C6D07AD221929B321D5CCF7,2023-05-14.22.19 Carson Ranked Doubles Loss,eurostadium_night_p,Mannfield (Night),ranked-doubles,Ranked Doubles,340,False,...,https://avatars.akamai.steamstatic.com/888251e...,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,86a2b97b-ba2d-4745-8d6f-da7f90807047,https://ballchasing.com/api/replays/86a2b97b-b...,A274331D4AC687AD9AA5D0A7E98D89A1,2023-05-14.16.19 EnragedPanda Ranked Doubles Win,trainstation_night_p,Urban Central (Night),ranked-doubles,Ranked Doubles,209,False,...,https://avatars.akamai.steamstatic.com/fef49e7...,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,30cd30a9-ece0-49dd-890f-e0ad652df7fa,https://ballchasing.com/api/replays/30cd30a9-e...,C83A945D48A109932CD51BBD375826D1,2023-05-15.00.19 Julpoo Ranked Doubles Loss,eurostadium_night_p,Mannfield (Night),ranked-doubles,Ranked Doubles,318,False,...,https://avatars.akamai.steamstatic.com/a5371d8...,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data['id'].unique()
data[data['id'] == 'd5a78371-fa71-46d7-8b10-11b8a64de5f8']

,id,link,rocket_league_id,replay_title,map_code,map_name,playlist_id,playlist_name,duration,overtime,...,uploader.avatar,blue.goals,orange.goals,overtime_seconds,blue_team,blue_player1,blue_player2,orange_team,orange_player1,orange_player2
46,d5a78371-fa71-46d7-8b10-11b8a64de5f8,https://ballchasing.com/api/replays/d5a78371-f...,BAF69C2646D1BD08FDA27295D260DE26,2023-05-13.23.56 R W 2 Ranked Doubles Loss,eurostadium_p,Mannfield,ranked-doubles,Ranked Doubles,206,False,...,https://avatars.akamai.steamstatic.com/5ed138c...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
data.columns

Index(['id', 'link', 'rocket_league_id', 'replay_title', 'map_code',
       'map_name', 'playlist_id', 'playlist_name', 'duration', 'overtime',
       'season', 'season_type', 'date', 'date_has_tz', 'visibility', 'created',
       'min_rank.tier', 'min_rank.division', 'min_rank.name', 'min_rank.id',
       'max_rank.tier', 'max_rank.division', 'max_rank.name', 'max_rank.id',
       'uploader.steam_id', 'uploader.name', 'uploader.profile_url',
       'uploader.avatar', 'blue.goals', 'orange.goals', 'overtime_seconds',
       'blue_team', 'blue_player1', 'blue_player2', 'orange_team',
       'orange_player1', 'orange_player2'],
      dtype='object')

In [54]:
get_players_csv('40f64128-d24c-4fe3-a959-c6805fb9e29d')[0]

color                                      blue
team name                                   NaN
player name                                Neon
platform                                  Steam
player id                     76561198196407241
                                    ...        
percentage neutral third                  29.76
time offensive third                      31.89
percentage offensive third                 9.67
demos inflicted                               2
demos taken                                   2
Name: 0, Length: 72, dtype: object

In [13]:
get_teams_csv('40f64128-d24c-4fe3-a959-c6805fb9e29d').T.columns

Index(['team name', 'game duration', 'score', 'goals', 'assists', 'saves',
       'shots', 'shots conceded', 'goals conceded', 'shooting percentage',
       'bpm', 'avg boost amount', 'amount collected',
       'amount collected big pads', 'amount collected small pads',
       'count collected big pads', 'count collected small pads',
       'amount stolen', 'amount stolen big pads', 'amount stolen small pads',
       'count stolen big pads', 'count stolen small pads', '0 boost time',
       '100 boost time', 'amount used while supersonic',
       'amount overfill total', 'amount overfill stolen', 'total distance',
       'time slow speed', 'time boost speed', 'time supersonic speed',
       'time on ground', 'time low in air', 'time high in air',
       'time powerslide', 'count powerslide', 'time behind ball',
       'time in front of ball', 'time defensive half', 'time offensive half',
       'time defensive third', 'time neutral third', 'time offensive third',
       'time ball poss

In [40]:
a = add_team_data(data)

In [58]:
b = add_player_data(a)

In [65]:
a['blue_player1'][0]

color                                                     blue
team name                                                  NaN
player name                                          Howfy M.a
platform                                                  Epic
player id                     424bad8ccb27464bb8d77a5ff887105b
                                            ...               
percentage neutral third                                 30.68
time offensive third                                     70.64
percentage offensive third                               17.82
demos inflicted                                              0
demos taken                                                  1
Name: 0, Length: 72, dtype: object